# Import necessary libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy.spatial.distance import cosine
from transformers import BertTokenizer, BertModel, TFBertModel, AutoModel, AutoTokenizer, TFAutoModel, BertConfig, AutoConfig
import tensorflow as tf
import torch
from ast import literal_eval
import warnings
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
warnings.filterwarnings('ignore')

In [ ]:
# !pip install transformers

In [ ]:
# !pip install torch torchvision

In [ ]:
# download nltk packages
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

# User input

In [ ]:
cuisine_city_input = "japanese food amsterdam"
test_categorical_features = tf.constant([["japanese food amsterdam"]], dtype=tf.string)
test_numerical_features = tf.constant([[5, 500.0, 100.0]])
review_input = "delicious sea food"

# Process cuisine and city

In [ ]:
cuisine_city_df = pd.read_csv(r"/content/drive/MyDrive/Datasets/SAC/after_cuisine_style_preprocessing2.csv", delimiter=',')
nRow, nCol = cuisine_city_df.shape
print(f'There are {nRow} rows and {nCol} columns')

There are 125527 rows and 11 columns


In [ ]:
cuisine_city_df = cuisine_city_df.drop(columns=['Unnamed: 0',
                                                'Ranking',
                                                'Rating',
                                                'Price Range',
                                                'Number of Reviews',
                                                'Reviews',
                                                'URL_TA',
                                                'ID_TA'])
cuisine_city_df['Cuisine_Style_City'] = cuisine_city_df['Cuisine Style'] + ' ' + cuisine_city_df['City']
# cuisine_city_df

,Name,City,Cuisine Style,Cuisine_Style_City
0,Martine of Martine's Table,Amsterdam,French Dutch European,French Dutch European Amsterdam
1,De Silveren Spiegel,Amsterdam,Dutch European Vegetarian Friendly Gluten Free Options,Dutch European Vegetarian Friendly Gluten Free Options Amsterdam
2,La Rive,Amsterdam,Mediterranean French International European Vegetarian Friendly Vegan Options,Mediterranean French International European Vegetarian Friendly Vegan Options Amsterdam
3,Vinkeles,Amsterdam,French European International Contemporary Vegetarian Friendly Vegan Options Gluten Free Options,French European International Contemporary Vegetarian Friendly Vegan Options Gluten Free Options Amsterdam
4,Librije's Zusje Amsterdam,Amsterdam,Dutch European International Vegetarian Friendly Vegan Options Gluten Free Options,Dutch European International Vegetarian Friendly Vegan Options Gluten Free Options Amsterdam
...,...,...,...,...
125522,Konrad Kaffee- & Cocktailbar,Zurich,European Vegetarian Friendly Options Mediterranean,European Vegetarian Friendly Options Mediterranean Zurich
125523,Blueberry American Bakery,Zurich,Cafe,Cafe Zurich
125524,Restaurant Bahnhof,Zurich,European Vegetarian Friendly Options Mediterranean,European Vegetarian Friendly Options Mediterranean Zurich
125525,Yoyo Pizza,Zurich,Fast Food,Fast Food Zurich


In [ ]:
tz = BertTokenizer.from_pretrained("bert-base-cased")
model = BertModel.from_pretrained('bert-base-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [ ]:
model_name = "bert-base-multilingual-cased"

In [ ]:
config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [ ]:
x = tokenizer(
    text=cuisine_city_df['Cuisine_Style_City'].to_list(),
    add_special_tokens=True,
    max_length=512,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)

In [ ]:
model = TFAutoModel.from_pretrained(model_name)

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
all_data_set_tokenized = pd.read_pickle("/content/drive/MyDrive/Datasets/SAC/pickle1.pkl")

## Processed user input

In [ ]:
inputs1 = tokenizer(cuisine_city_input, return_tensors="tf")
output1 = model(inputs1)
# output1

TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=<tf.Tensor: shape=(1, 8, 768), dtype=float32, numpy=
array([[[-0.12098752, -0.2569213 ,  0.04764283, ...,  0.26020035,
          0.03366632,  0.01059289],
        [-0.13651267, -0.316182  ,  0.1343944 , ..., -0.17188333,
         -0.29034752,  0.17484212],
        [-0.2775982 , -0.00501321, -0.28001946, ...,  0.44077623,
         -0.72724605,  0.04474235],
        ...,
        [-0.4107216 , -1.5342523 , -0.4639897 , ...,  0.6980805 ,
          0.2691051 , -0.43122098],
        [-0.13401592, -0.7123925 ,  0.16709189, ..., -0.38587183,
          0.10707016, -1.226602  ],
        [-0.5583134 , -0.558529  ,  0.2907887 , ...,  0.11025342,
          0.08617856,  0.19166283]]], dtype=float32)>, pooler_output=<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
array([[ 1.85694098e-01, -5.74205890e-02,  2.46006936e-01,
        -2.20676675e-01, -2.91819405e-02,  3.08109045e-01,
         3.08930576e-01,  1.33042797e-01, -3.42351973e-0

## Processed dataset

In [ ]:
output2 = all_data_set_tokenized

In [ ]:
lastHidden1 = np.squeeze(output1['last_hidden_state'], axis=0)
lastHidden1 = lastHidden1[0]

cosine_similarities = []
for data_output in output2:
    lastHidden2 = np.squeeze(data_output['last_hidden_state'], axis=0)
    lastHidden2 = lastHidden2[0]
    cosine_similarities.append(1 - cosine(lastHidden1, lastHidden2))

cuisine_city_df['Scores'] = cosine_similarities
# cuisine_city_df

,Name,City,Cuisine Style,Cuisine_Style_City,Scores
0,Martine of Martine's Table,Amsterdam,French Dutch European,French Dutch European Amsterdam,0.829087
1,De Silveren Spiegel,Amsterdam,Dutch European Vegetarian Friendly Gluten Free Options,Dutch European Vegetarian Friendly Gluten Free Options Amsterdam,0.860450
2,La Rive,Amsterdam,Mediterranean French International European Vegetarian Friendly Vegan Options,Mediterranean French International European Vegetarian Friendly Vegan Options Amsterdam,0.861112
3,Vinkeles,Amsterdam,French European International Contemporary Vegetarian Friendly Vegan Options Gluten Free Options,French European International Contemporary Vegetarian Friendly Vegan Options Gluten Free Options Amsterdam,0.870836
4,Librije's Zusje Amsterdam,Amsterdam,Dutch European International Vegetarian Friendly Vegan Options Gluten Free Options,Dutch European International Vegetarian Friendly Vegan Options Gluten Free Options Amsterdam,0.879691
...,...,...,...,...,...
125522,Konrad Kaffee- & Cocktailbar,Zurich,European Vegetarian Friendly Options Mediterranean,European Vegetarian Friendly Options Mediterranean Zurich,0.649154
125523,Blueberry American Bakery,Zurich,Cafe,Cafe Zurich,0.777685
125524,Restaurant Bahnhof,Zurich,European Vegetarian Friendly Options Mediterranean,European Vegetarian Friendly Options Mediterranean Zurich,0.649154
125525,Yoyo Pizza,Zurich,Fast Food,Fast Food Zurich,0.797013


# Process price range, rating, and number of reviews

In [ ]:
"""Recommander_system_combined_models.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1mq-Xu76f2SO4RpOooGWOvX8eLCDLmkz-

# Preprocessing of TA_restaurants_curated.csv
"""


restaurants_df = pd.read_csv("/content/drive/MyDrive/Datasets/SAC/TA_restaurants_curated.csv")

"""## Drop unnecessary columns"""

restaurants_df = restaurants_df.drop([restaurants_df.columns[0],
                                      restaurants_df.columns[3],
                                      restaurants_df.columns[4],
                                      restaurants_df.columns[8],
                                      restaurants_df.columns[9],
                                      restaurants_df.columns[10]], axis=1)

"""## Fill all NaN values"""

fill_values = {"Rating":restaurants_df.value_counts("Rating").index[0],
                "Price Range":restaurants_df.value_counts("Price Range").index[0],
                "Number of Reviews":restaurants_df.value_counts("Number of Reviews").index[0]}

restaurants_df = restaurants_df.fillna(value=fill_values)

"""## Convert `Price Range` column values from string to int"""

column_values = list(zip(restaurants_df.value_counts("Price Range").index, [50, 5, 500]))

restaurants_df = restaurants_df.replace(to_replace={"Price Range":{column_values[0][0]:column_values[0][1],
                                                                   column_values[1][0]:column_values[1][1],
                                                                   column_values[2][0]:column_values[2][1]}})

"""## Convert `Rating` value -1 to 1"""

column_values = list(zip([restaurants_df.value_counts("Rating").index[-1]], [1]))

restaurants_df = restaurants_df.replace(to_replace={"Rating":{column_values[0][0]:column_values[0][1]}})

"""## Adding `Recommandation` column as labels for the model"""

# create a list of conditions
conditions = [(restaurants_df['Rating'] < 4),
              (restaurants_df['Rating'] >= 4)]

# create a list of the values to assign for each condition
values = [0, 1]

# create a new column and assign values based on conditions
restaurants_df['Recommandation'] = np.select(conditions, values)

"""# Prepare dataset

## Get numerical features
"""

# convert numerical features DataFrame to TensorFlow tensor
train_numerical_features = tf.convert_to_tensor(restaurants_df[['Rating', 'Price Range', 'Number of Reviews']])

# convert labels DataFrame to TensorFlow tensor
train_labels = tf.convert_to_tensor(restaurants_df['Recommandation'])

# normalize the features
# normalizer = StandardScaler()
normalizer = MinMaxScaler()
train_numerical_features = normalizer.fit_transform(train_numerical_features)

"""## Get categorical features"""

categorical = []
train_categorical_features = np.array(restaurants_df[["Name", "City"]])

for values in train_categorical_features:
    categorical.append(' '.join(values))

train_categorical_features = np.expand_dims(np.array(categorical), axis=1)

"""## Create dataset"""

# turn train data into TensorFlow DataSet
train_dataset_features = tf.data.Dataset.from_tensor_slices((train_numerical_features,
                                                             train_categorical_features,
                                                             train_numerical_features))
train_dataset_labels = tf.data.Dataset.from_tensor_slices(train_labels)
train_dataset_concatenated = tf.data.Dataset.zip((train_dataset_features, train_dataset_labels))

# take the TensorSliceDatasets and turn them into prefetched datasets
train_dataset_concatenated = train_dataset_concatenated.batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

"""# Recreating Google's Wide and Deep model

## Create Tokenization layer
"""

# get the number of words in each sentence from the training dataset
feature_length = [len(feature.split()) for feature in train_categorical_features[:, 0]]

# get the average length of sentences
max_length = int(np.max(feature_length))
print(f"Maximum number of words is {max_length}")

# get total unique words
total_unique_words = set([word for sentence in train_categorical_features[:, 0] for word in sentence.split()])

Maximum number of words is 14


In [ ]:
# setup TextVectorization variables
max_vocabulary_length = len(total_unique_words)
max_sequence_length = max_length

word_text_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=max_vocabulary_length, # number of words in the vocabulary (if set to None it will automatically get the number of unique tokens)
    standardize="lower_and_strip_punctuation",
    split="whitespace",
    ngrams=None, # create groups of n-words
    output_mode="int", # map tokens to numbers
    output_sequence_length=max_sequence_length, # the length of the sequences
    pad_to_max_tokens=True # get the length of the longest sequence and add 0s to the end of shorter sequences
)

# fit the training dataset to the text vectorizer and get the vocabulary
word_text_vectorizer.adapt(train_categorical_features)
vocabulary = word_text_vectorizer.get_vocabulary()

"""## Create Emdedding layer"""

# creating embedding layer
word_embedding = tf.keras.layers.Embedding(input_dim=len(vocabulary),
                                           output_dim=3,
                                           input_length=max_sequence_length,
                                           embeddings_initializer="uniform")

"""## Defining the recommander system"""

# define input layers
wide_inputs = tf.keras.layers.Input(shape=(3,), name='wide_inputs')
deep_numerical_inputs = tf.keras.layers.Input(shape=(3,), name='deep_numerical_inputs')
deep_categorical_inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.string, name='deep_categorical_inputs')

# define wide component
wide_layer = tf.keras.layers.Dense(1, activation='sigmoid', name='wide_output')(wide_inputs)

# define tokenization and word embeddings
token_layer = word_text_vectorizer(deep_categorical_inputs)
embed_layer = word_embedding(token_layer)

# define concatenate layer for numerical and categorical features
combined_layer = tf.keras.layers.Concatenate(axis=1)([embed_layer, tf.expand_dims(deep_numerical_inputs, axis=1)])

# define deep component
deep_layer = tf.keras.layers.Dense(64, activation='relu')(combined_layer)
deep_layer = tf.keras.layers.Dense(64, activation='relu')(deep_layer)
deep_layer = tf.keras.layers.Dense(64, activation='relu')(deep_layer)
deep_layer = tf.keras.layers.Dense(64, activation='relu')(deep_layer)
deep_layer = tf.keras.layers.Dense(1, activation='sigmoid', name='deep_output')(deep_layer)

# combine wide and deep components
combined_layer = tf.keras.layers.Concatenate(axis=1)([tf.expand_dims(wide_layer, axis=1), deep_layer])

# define output layer
output_layer = tf.keras.layers.Dense(1, activation='sigmoid', name='output_layer')(tf.squeeze(combined_layer, axis=-1))

# create the final model
model = tf.keras.models.Model(inputs=[wide_inputs,
                                      deep_categorical_inputs,
                                      deep_numerical_inputs], outputs=output_layer, name="wide_and_deep")

# compile the model with label smoothing
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.3), # log loss
              metrics=['accuracy'])

model.fit(train_dataset_concatenated,
          epochs=5,
          steps_per_epoch=len(train_dataset_concatenated),
          verbose=1)

model.evaluate(train_dataset_concatenated)

model_predictions = model.predict(train_dataset_concatenated)

model_predictions = tf.squeeze(model_predictions).numpy()

In [ ]:
"""# Testing the model"""

test_labels = []

for rating in test_numerical_features[:,0]:
    if rating < 4:
        test_labels.append(0)
    else:
        test_labels.append(1)

test_labels = tf.constant(test_labels)

test_numerical_features = normalizer.transform(test_numerical_features)
test_numerical_features = tf.expand_dims(test_numerical_features, axis=1)

test_data_features = tf.data.Dataset.from_tensor_slices((test_numerical_features, test_categorical_features, test_numerical_features))
test_labels = tf.data.Dataset.from_tensor_slices(test_labels)
test_data_concatenated = tf.data.Dataset.zip((test_data_features, test_labels))

test_prediction = model.predict(test_data_concatenated)
# test_prediction

"""## Testing Googles's Wide and Deep Recommander System"""

# restaurants_df["Scores"] = model_predictions.reshape((len(model_predictions),))

# restaurants_result_df = restaurants_df.sort_values(by=["Scores"], ascending=False)

# recommandations = []

# for prediction in test_prediction:
#     recommandations.append(restaurants_result_df[restaurants_result_df["Scores"] <= np.squeeze(prediction, axis=0)].head(10))
#     print(restaurants_result_df[restaurants_result_df["Scores"] <= np.squeeze(prediction, axis=0)].head(3))

1/1 [==============================] - 0s 37ms/step
[[0.8656888]]
                               Name    City  Rating  Price Range  \
19682              Yoko Sushi Mitte  Berlin     5.0           50   
119668  Wein&Vino - Croatia Premium  Vienna     5.0           50   
111602                     Fryttati    Rome     5.0            5   

        Number of Reviews  Recommandation    Scores  
19682                 2.0               1  0.865687  
119668                2.0               1  0.865687  
111602               14.0               1  0.865684  


# Process reviews (preferences)

In [ ]:
reviews_df = pd.read_csv(r"/content/drive/MyDrive/Datasets/SAC/TA_restaurants_curated.csv")
reviews_df.drop(columns=['Unnamed: 0',
                         'URL_TA',
                         'ID_TA',
                         'Cuisine Style',
                         'Price Range',
                         'Ranking',
                         'Number of Reviews',
                         'Rating'], inplace=True)
# reviews_df

,Name,City,Reviews
0,Martine of Martine's Table,Amsterdam,"[['Just like home', 'A Warm Welcome to Wintry Amsterdam'], ['01/03/2018', '01/01/2018']]"
1,De Silveren Spiegel,Amsterdam,"[['Great food and staff', 'just perfect'], ['01/06/2018', '01/04/2018']]"
2,La Rive,Amsterdam,"[['Satisfaction', 'Delicious old school restaurant'], ['01/04/2018', '01/04/2018']]"
3,Vinkeles,Amsterdam,"[['True five star dinner', 'A superb evening of fine dining, hospitali...'], ['12/20/2017', '12/17/2017']]"
4,Librije's Zusje Amsterdam,Amsterdam,"[['Best meal.... EVER', 'super food experience'], ['01/06/2018', '01/04/2018']]"
...,...,...,...
125522,Konrad Kaffee- & Cocktailbar,Zurich,NaN
125523,Blueberry American Bakery,Zurich,NaN
125524,Restaurant Bahnhof,Zurich,NaN
125525,Yoyo Pizza,Zurich,NaN


In [ ]:
def convert_to_list(x):
    try:
        return literal_eval(x)
    except ValueError:
        return pd.NA

def process_sentences(text):
    temp_sent =[]

    words = nltk.word_tokenize(text)

    tags = nltk.pos_tag(words)
    for i, word in enumerate(words):
        if tags[i][1] in ('VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'):
            lemmatized = lemmatizer.lemmatize(word, 'v')
        else:
            lemmatized = lemmatizer.lemmatize(word)

        if lemmatized not in stop_words and lemmatized.isalpha():
            temp_sent.append(lemmatized)

    full_sentence = ' '.join(temp_sent)
    full_sentence = full_sentence.replace("n't", " not")
    full_sentence = full_sentence.replace("'m", " am")
    full_sentence = full_sentence.replace("'s", " is")
    full_sentence = full_sentence.replace("'re", " are")
    full_sentence = full_sentence.replace("'ll", " will")
    full_sentence = full_sentence.replace("'ve", " have")
    full_sentence = full_sentence.replace("'d", " would")

    return full_sentence

def recommend(df, description):
    description = description.lower()

    description = process_sentences(description)
    description = description.strip()

    df['Recommandation'] = df['Reviews'] + ' ' + df['City']

    tfidfvec = TfidfVectorizer()

    vec = tfidfvec.fit(df["Recommandation"].values.astype('U'))

    features = vec.transform(df["Recommandation"].values.astype('U'))
    description_vector =  vec.transform([description])

    cos_sim = linear_kernel(description_vector, features)

    df['Scores'] = cos_sim[0]

    return df

def complete_NA(x):
    if x is not pd.NA:
       return ', '.join(x[0])
    else:
        return ''


reviews_df['Reviews'] = reviews_df['Reviews'].apply(convert_to_list)

reviews_df['Reviews'] = reviews_df['Reviews'].apply(complete_NA)
reviews_df['Reviews'] = reviews_df['Reviews'].str.lower()
reviews_df['City'] = reviews_df['City'].str.lower()

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

review_result_df = recommend(reviews_df, review_input)
# review_result_df

,Name,City,Reviews,Recommandation,Scores
0,Martine of Martine's Table,amsterdam,"just like home, a warm welcome to wintry amsterdam","just like home, a warm welcome to wintry amsterdam amsterdam",0.000000
1,De Silveren Spiegel,amsterdam,"great food and staff, just perfect","great food and staff, just perfect amsterdam",0.061812
2,La Rive,amsterdam,"satisfaction, delicious old school restaurant","satisfaction, delicious old school restaurant amsterdam",0.118793
3,Vinkeles,amsterdam,"true five star dinner, a superb evening of fine dining, hospitali...","true five star dinner, a superb evening of fine dining, hospitali... amsterdam",0.000000
4,Librije's Zusje Amsterdam,amsterdam,"best meal.... ever, super food experience","best meal.... ever, super food experience amsterdam",0.055112
...,...,...,...,...,...
125522,Konrad Kaffee- & Cocktailbar,zurich,,zurich,0.000000
125523,Blueberry American Bakery,zurich,,zurich,0.000000
125524,Restaurant Bahnhof,zurich,,zurich,0.000000
125525,Yoyo Pizza,zurich,,zurich,0.000000


# Extract ***n*** number of recommandations

In [ ]:
final_df = cuisine_city_df
final_df.insert(loc=4, column='Rating', value=restaurants_df['Rating'])
final_df.insert(loc=5, column='Price Range', value=restaurants_df['Price Range'])
final_df.insert(loc=6, column='Number of Reviews', value=restaurants_df['Number of Reviews'])
final_df.insert(loc=7, column='Reviews', value=review_result_df['Reviews'])
final_df['Scores'] = (cuisine_city_df['Scores'] + review_result_df['Scores'] + test_prediction[0])/3
# final_df

,Name,City,Cuisine Style,Cuisine_Style_City,Rating,Price Range,Number of Reviews,Reviews,Scores
0,Martine of Martine's Table,Amsterdam,French Dutch European,French Dutch European Amsterdam,5.0,50,136.0,"just like home, a warm welcome to wintry amsterdam",0.564925
1,De Silveren Spiegel,Amsterdam,Dutch European Vegetarian Friendly Gluten Free Options,Dutch European Vegetarian Friendly Gluten Free Options Amsterdam,4.5,500,812.0,"great food and staff, just perfect",0.595984
2,La Rive,Amsterdam,Mediterranean French International European Vegetarian Friendly Vegan Options,Mediterranean French International European Vegetarian Friendly Vegan Options Amsterdam,4.5,500,567.0,"satisfaction, delicious old school restaurant",0.615198
3,Vinkeles,Amsterdam,French European International Contemporary Vegetarian Friendly Vegan Options Gluten Free Options,French European International Contemporary Vegetarian Friendly Vegan Options Gluten Free Options Amsterdam,5.0,500,564.0,"true five star dinner, a superb evening of fine dining, hospitali...",0.578842
4,Librije's Zusje Amsterdam,Amsterdam,Dutch European International Vegetarian Friendly Vegan Options Gluten Free Options,Dutch European International Vegetarian Friendly Vegan Options Gluten Free Options Amsterdam,4.5,500,316.0,"best meal.... ever, super food experience",0.600164
...,...,...,...,...,...,...,...,...,...
125522,Konrad Kaffee- & Cocktailbar,Zurich,European Vegetarian Friendly Options Mediterranean,European Vegetarian Friendly Options Mediterranean Zurich,4.0,50,2.0,,0.504948
125523,Blueberry American Bakery,Zurich,Cafe,Cafe Zurich,4.0,50,2.0,,0.547791
125524,Restaurant Bahnhof,Zurich,European Vegetarian Friendly Options Mediterranean,European Vegetarian Friendly Options Mediterranean Zurich,4.0,50,2.0,,0.504948
125525,Yoyo Pizza,Zurich,Fast Food,Fast Food Zurich,4.0,50,2.0,,0.554234


In [ ]:
n_recommandations = 3
final_df = final_df.sort_values(by=["Scores"], ascending=False)
final_result = final_df.head(n_recommandations)
# final_result

,Name,City,Cuisine Style,Cuisine_Style_City,Rating,Price Range,Number of Reviews,Reviews,Scores
113854,Ristorante Oasi Marina,Rome,Seafood Delicatessen Healthy Mediterranean Gluten Free Options,Seafood Delicatessen Healthy Mediterranean Gluten Free Options Rome,4.5,50,58.0,delicious sea food,0.865700
113120,L'Officina del Pesce,Rome,Italian Seafood,Italian Seafood Rome,4.5,500,30.0,delicious sea food,0.858171
41966,Grilled and company,Lisbon,Mediterranean European Portuguese Vegetarian Friendly Vegan Options,Mediterranean European Portuguese Vegetarian Friendly Vegan Options Lisbon,4.0,50,416.0,"good sea food, good sea food restaurant",0.835985


In [ ]:
get_results = final_result[['Name', 'City', 'Cuisine Style', 'Price Range', 'Reviews']]
get_results = get_results.to_dict(orient='list')
get_results = list(zip(*list(get_results.values())))
# get_results

[('Ristorante Oasi Marina',
  'Rome',
  'Seafood Delicatessen Healthy Mediterranean Gluten Free Options',
  50,
  'delicious sea food'),
 ("L'Officina del Pesce",
  'Rome',
  'Italian Seafood',
  500,
  'delicious sea food'),
 ('Grilled and company',
  'Lisbon',
  'Mediterranean European Portuguese Vegetarian Friendly Vegan Options',
  50,
  'good sea food, good sea food restaurant')]